**Importing files and looking at structures**

In [1]:
import pandas as pd
import numpy as np

In [2]:
overall_pop = pd.read_csv('~/720/pds2021-opioids-team-8-1/20_intermediate_files/Population_2000-2019.csv')

In [3]:
overall_cause_of_death = pd.read_csv('~/720/pds2021-opioids-team-8-1/20_intermediate_files/Underlying Cause of Death, 2003-2015.csv')

In [ ]:
overall_cause_of_death.sample(15)

In [ ]:
overall_pop.sample(15)

In [ ]:
overall_pop.columns

**Now we need to get both datasets into a form where we can merge on state-county-year**

**It looks like the cause of death data has *state* included in the *county* column, but the population has seperate *county* and *state* columns. So we need to separate the county-state information in the cause of death dataset**

In [4]:
# extract state information from county column in the cause of death data

overall_cause_of_death['State'] = ''
overall_cause_of_death.State = [overall_cause_of_death.County[i][-2:] for i in range(len(overall_cause_of_death.County))]

In [ ]:
overall_cause_of_death.sample(10)

In [5]:
# extract county information from county column in the cause of death data

overall_cause_of_death['temp_County'] = ''
overall_cause_of_death['temp_County']= [overall_cause_of_death.County[i][:-4] for i in range(len(overall_cause_of_death.County))]

In [ ]:
overall_cause_of_death.sample(10)

**We now have seperate *state* and *county* columns in the cause of death data set, so we drop the County column (and other unnecessary columns for merging) and rename the temp_County column**

In [6]:
overall_cause_of_death.drop(labels=['County', 'Unnamed: 0', 'Notes', 'Year Code'], axis=1, inplace=True)

In [7]:
overall_cause_of_death.rename({'temp_County':'County'}, axis=1, inplace=True)

In [ ]:
overall_cause_of_death.sample(10)

**Now must melt down year headers from the population data set into single column of observations to match the cause of death data set**

In [8]:
# first drop unnecessary columns "Unnamed: 0" and "_merge"
overall_pop.drop(labels=['Unnamed: 0', '_merge'], axis=1, inplace=True)

In [ ]:
overall_pop.head()

In [9]:
overall_pop_melted = overall_pop.melt(id_vars=['County', 'State'], value_vars=overall_pop.filter(like='20').columns.tolist(), var_name='Year', value_name='Population')

In [ ]:
overall_pop_melted.head()

In [ ]:
overall_cause_of_death.head(10)

In [ ]:
print(overall_cause_of_death.dtypes)
print(overall_pop_melted.dtypes)

**Change types of columns we want to merge so they match**

In [11]:
overall_pop_melted.Year = overall_pop_melted.Year.astype(np.float64)

In [ ]:
overall_pop_melted.dtypes

**Cause of death data is now in a format to merge successfully with the population dataset**

In [12]:
merged = overall_cause_of_death.merge(overall_pop_melted, on=['Year', 'State', 'County'], validate='m:1', indicator=True)

In [13]:
assert merged[merged['_merge'] != 'both'].empty

In [14]:
merged.isnull().sum().sum()

0

In [15]:
assert [not merged[col].isnull().any() for col in merged.columns]

In [ ]:
merged.sample(30)

In [ ]:
merged['Drug/Alcohol Induced Cause'].values

In [ ]:
merged['Drug/Alcohol Induced Cause Code'].unique()

In [16]:
overdose_deaths = merged[(merged['Drug/Alcohol Induced Cause Code'] == 'D1') | (merged['Drug/Alcohol Induced Cause Code'] == 'D2') | (merged['Drug/Alcohol Induced Cause Code'] == 'D4')].copy()

In [22]:
overdose_deaths['Total Deaths By Overdose'] =  overdose_deaths.groupby(['State', 'County', 'Year'])['Deaths'].transform(np.sum)

In [27]:
overdose_deaths.drop_duplicates(subset=['Year', 'State', 'County', 'Total Deaths By Overdose'], inplace=True)

In [29]:
overdose_deaths.drop(labels=['Drug/Alcohol Induced Cause', 'Drug/Alcohol Induced Cause Code', 'Deaths', '_merge'], axis=1, inplace=True)